## First, run this cell, then run the main function scroll down to see it.

In [ ]:
# First run this cell to import 
import pandas as pd # pandas
import numpy as np  # numpy
import plotly.graph_objects as go # plotly
from plotly.offline import plot   
import matplotlib.pyplot as plt   # matplotlib
import datetime 
from pycoingecko import CoinGeckoAPI # API
from mplfinance.original_flavor import candlestick2_ohlc
    

In [ ]:
# Installing 
def check_requirements():
    print("\nChecking requirements ..... Wait !")
    # !pip install pycoingecko  
    # !pip install plotly
    # !pip install mplfinance
    # !pip install --upgrade nbformat
    return True


<h2 id="ref1">REST APIs</h2>


Rest API’s function by sending a request,  the request is communicated via HTTP message. The HTTP message usually contains a JSON file. This contains instructions for what operation we would like the service or resource to perform. In a similar manner, API returns a response, via an HTTP message, this response is usually contained within a JSON.

In cryptocurrency a popular method to display the movements of the price of a currency.

![Example Candlestick Graph](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/candlestick.png)

Here is a description of the candle sticks.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/explanation.png" alt="Candle Stick Definition" width="800" height="600">

In this, we will be using the to create one of these candlestick graphs for Bitcoin. We will use the API to get the price data for 30 days with 24 observation per day, 1 per hour. We will find the max, min, open, and close price per day meaning we will have 30 candlesticks and use that to generate the candlestick graph. Although we are using the CoinGecko API we will use a Python client/wrapper for the API called <a href=https://github.com/man-c/pycoingecko>PyCoinGecko</a>. PyCoinGecko will make performing the requests easy and it will deal with the enpoint targeting.


In [ ]:
# cg = CoinGeckoAPI()

# bitcoin_data = cg.get_coin_market_chart_by_id(id='bitcoin', vs_currency='usd', days=30)
def start(coin_inp:str, currency:str, days_inp:int):
    cg = CoinGeckoAPI()
    bitcoin_data = cg.get_coin_market_chart_by_id(id = coin_inp, vs_currency = currency, days = days_inp)
    filter(bitcoin_data,coin_inp)

The response we get is in the form of a JSON which includes the price, market caps, and total volumes along with timestamps for each observation. We are focused on the prices so we will select that data.


In [ ]:
# bitcoin_price_data = bitcoin_data['prices']

# bitcoin_price_data[0:5]
def filter(bitcoin_data,coin_inp):
    bitcoin_price_data = bitcoin_data['prices']
    # bitcoin_price_data[0:5]
    con_to_dataframe(bitcoin_price_data, coin_inp)

Finally lets turn this data into a Pandas DataFrame.


In [ ]:
def con_to_dataframe(bitcoin_price_data, coin_inp):
    data = pd.DataFrame(bitcoin_price_data, columns=['Timestamp','prices'])
    
    # converting timestamp to date
    data['Date'] = pd.to_datetime(data['Timestamp'], unit = 'ms')
    data['Date_only'] = pd.to_datetime(data['Date']).dt.date
    # print(data.head())
    candlestick(data, coin_inp)

Now that we have the DataFrame we will convert the timestamp to datetime and save it as a column called <code>Date</code>. We will map our <code>unix_to_datetime</code> to each timestamp and convert it to a readable datetime.


Using this modified dataset we can now group by the <code>Date</code> and find the min, max, open, and close for the candlesticks.


In [ ]:
def candlestick(data, coin_inp):
    candlestick_data = data.groupby('Date_only', as_index=False).agg({"prices": ['min', 'max', 'first', 'last']})  # sub col of Price as min, max, first, last --> function_names # In short we have divided pandas col into sub columns ( dataframes )
    figure_result = figure(candlestick_data, data)
    show_figure(figure_result, coin_inp)
    


# some testing

# candlestick_data['prices'] # --> see, prices col has now its sub columns min, max, first, last
# # type(candlestick_data['prices']) # --> dataframe
# # candlestick_data.shape
# candlestick_data['prices'].shape # --> (31, 4) where 31 are rows and 4 cols i.e (min, max, first, last)
 
# # checking
# # candlestick_data['prices'].loc[0:5,'min':'max']


Finally we are now ready to use plotly to create our Candlestick Chart.


In [ ]:
def figure(candlestick_data, data_only):
    fig = go.Figure(data=[go.Candlestick(x = candlestick_data['Date_only'],
                open=candlestick_data['prices']['first'], 
                high=candlestick_data['prices']['max'],
                low=candlestick_data['prices']['min'], 
                close=candlestick_data['prices']['last'])
                ])
    return fig

def show_figure(fig, coin_inp):
    print("\nThe figure is as follow")
    fig.update_layout(xaxis_rangeslider_visible=False, xaxis_title = 'Date',yaxis_title = 'Price (USD $)',title = f'{coin_inp.capitalize()} Candlestick chart')
    fig.show()    
    


## Main function

In [ ]:
def main():
    try:
        while True:
            coin_inp = input("Which coin data you want to see ?").lower()
            currency_inp = input("Choose currency:")
            days_input = int(input("Enter the no of days for price data:"))
            print("\nstarting....")
            break
        
        if check_requirements():
            start(coin_inp, currency_inp, days_input)
    except Exception as e:
        print(f"Error:{e}")
    
    
main()

# cg = CoinGeckoAPI()
# list = cg.get_coins_list()
# list
